## Setup

In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# get modules
from air_quality import *
from fisheries import *
from noise import *
from recreation import *
from scenic_resources import *
from soil_conservation import *
from transportation_sustainablecommunities import *
from vegetation_preservation import *
from water_quality import *
from wildlife import *

## Air Quality

In [ ]:
# get PM 2.5
df = get_airquality_data_sql()
# plot PM 2.5
plot_pm2_5_annual(df, draft=False)

In [ ]:
# get PM 2.5
df = get_airquality_data_sql()
# plot PM 2.5
plot_pm2_5_24hour(df, draft=False)

In [ ]:
# get PM 10
df = get_airquality_data_sql()
#Get PM 10 24 hour high
plot_pm10_24hr(df, draft= False)

In [ ]:
# get PM 10
df = get_airquality_data_sql()
#Plot PM 10 Average Annual Concentrations
plot_pm10_annual(df, draft= False)

In [ ]:
# get 03
df = get_airquality_data_sql()
#Plot 03 1 hour high
plot_o3_1hour_high(df, draft= False)

In [ ]:
# get CO
df = get_airquality_data_sql()
#Plot CO 1 hour high
plot_co_8hour_avg(df, draft= True)

In [ ]:
#Get 50th regionalVisibility Bliss 
df = get_airquality_data_sql()
#Plot Visibility at Bliss
plot_50_Bliss_vis(df, draft= False)

In [ ]:
#Get 90th regionalVisibility Bliss 
df = get_airquality_data_sql()
#Plot 90th percentile Visibility at Bliss
plot_90_Bliss_vis(df, draft= False)

In [ ]:
#Get 50th Visibility subregional slt
df = get_airquality_data_sql()
#Plot 50th Visibility subregional slt
plot_50_SLT_vis(df, draft= False)

In [ ]:
#Get 90th Visibility subregional slt
df = get_airquality_data_sql()
#Plot 90th Visibility subregional slt
plot_90_SLT_vis(df, draft= False)

In [ ]:
#Get NOx data
df = get_NOX_data_sql()
#Plot NOX
plot_NOx(df, draft= False)

In [ ]:
#Get winter traffic data
df = get_wintertraffic_data_sql()
plot_winter_traffic(df, draft= False)

In [ ]:
# mid lake dissolved nitrogen data and chart
df = get_midlake_dissolved_nitrogen_data()
plot_midlake_dissolved_nitrogen(df, draft=False)

## Fisheries

In [ ]:
df = get_fishhab()
plot_fishhab(df, draft=False)

## Noise

## Soil Conservation

In [5]:
dfImpChg, dfImp2019 = get_soil_conservation_data_sql()
dfImpOld = transform_soil_conservation_data(dfImpChg, dfImp2019)
dfImpNew = transform_new_coverage_data()
df = add_new_coverage(dfImpOld, dfImpNew)

In [ ]:
# plot 1A - soil conservation
plot_soil_conservation(df, landcap = '1A', draft=False)
plot_soil_conservation(df, landcap = '1B', draft=False)
plot_soil_conservation(df, landcap = '1C', draft=False)
plot_soil_conservation(df, landcap = '2', draft=False)
plot_soil_conservation(df, landcap = '3', draft=False)
plot_soil_conservation(df, landcap = '4', draft=False)
plot_soil_conservation(df, landcap = '5', draft=False)
plot_soil_conservation(df, landcap = '6', draft=False)
plot_soil_conservation(df, landcap = '7', draft=False)

In [3]:
# Get SEZ Data
df = get_SEZ_data_web()
# Make SEZ/Wetland Restoration Chart
plot_SEZ_scores(df,draft=False) 

## Vegetation

In [14]:
#Tahoe Yellow Cress Chart
df = get_TYC_data_sql()
plot_TYC(df, draft=False)

In [ ]:
# Veg Treatment Chart
df = get_data_forest_fuel()
plot_forest_fuel(df, draft=True)

In [32]:
# Old Growth
df = get_old_growth()
plot_old_growth(df, draft=True)

## Water Quality

In [ ]:
# secchi depth data and chart
df = get_secchi_data_sql()
# df = get_secchi_data_web()
plot_secchi_depth(df, draft=False)

In [ ]:
# watercraft inspections
df = get_watercraft_inspection_data_web()
plot_watercraft_inspections(df, draft=False)

In [ ]:
# get water quality data
df = get_waterquality_data()
# plot total nitrogetn
plot_total_nitrogen(df, draft=False)
plot_total_phosphorus(df, draft=False)
plot_suspended_sediment(df, draft=False)

In [ ]:
# get total load reduction data
df = get_phosphorus_load_reduction()
plot_phosphorus_load_reduction(df, draft=False)

In [ ]:
# get total load reduction data
df = get_nitrogen_load_reduction()
plot_nitrogen_load_reduction(df, draft=False)

In [ ]:
# get total load reduction data
df = get_sediment_load_reduction()
plot_sediment_load_reduction(df, draft=False)

In [ ]:
# periyphyton data and chart
df = get_periphyton_data()
plot_periphyton(df, draft=False)

In [ ]:
# nearshore turbidity data and chart
df = get_nearshore_turbidity_data()
plot_nearshore_turbidity(df, draft=True)

In [ ]:
df = get_ais_infestation_data_sql()
plot_ais_infestation(df, draft=False)

## Wildlife

In [ ]:
# goshawk data and chart
df = get_wildlife_data_web()
plot_goshawk_data(df, draft=False)

In [ ]:
# osprey data and chart
df = get_wildlife_data_web()
plot_osprey_data(df, draft=True)

In [ ]:
# bald eagle winter roost data and chart
df = get_wildlife_data_web()
plot_bald_eagle_winter(df, draft=False)

In [ ]:
df = get_bald_eagle_data_wt_estimate_sql()
plot_bald_eagle_winter_est(df, draft=False)

In [ ]:
df = get_peregrine_falcon_data_wt_estimate_sql()
plot_pfalcon_data_wt_estimate(df, draft=False)

In [ ]:
df = get_osprey_data_wt_estimate_sql()
plot_osprey_data_wt_estimate(df, draft=False)

In [ ]:
# peregine falcon data and chart
df = get_wildlife_data_web()
plot_falcon_data(df, draft=False)

In [ ]:
# bald eagle summer roost data and chart
df = get_wildlife_data_web()
plot_bald_eagle_summer(df, draft=False)

In [ ]:
#waterfowl data and chart
df = get_waterfowl_data_web()
plot_waterfowl_data(df, draft=False)